In [1]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import numpy

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt'

sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import clonality_analysis_util
import re
from scipy import stats
import math
import msi_analysis_utils

In [2]:
def count_number_of_times_allele_is_in_multiplet(maf, allele):
    casesWithAllele = set(maf[maf['allele'] == allele]['Tumor_Sample_Barcode'])
    if len(casesWithAllele) == 0:
        return None
    alleleMaf = maf[maf['Tumor_Sample_Barcode'].isin(casesWithAllele)]
    gene = allele.split('_')[0]
    otherOncogenicGeneAlleles = alleleMaf[(alleleMaf['oncogenic'].notnull()) & (alleleMaf['Hugo_Symbol'] == gene) &(alleleMaf['allele'] != allele)]
    return (1.0*len(set(otherOncogenicGeneAlleles['Tumor_Sample_Barcode'])))/len(casesWithAllele)

#lambda function that marks whether an allele is part of a multiplet
def is_allele_multiplet(maf, allele, tsb):
    gene = allele.split('_')[0]
    if maf[(maf['allele'] != allele) & (maf['oncogenic'].notnull()) & (maf['Hugo_Symbol'] == gene) & (maf['Tumor_Sample_Barcode'] == tsb)].shape[0] > 0:
        return 1
    else:
        return 0

In [ ]:
allImpactMuts = analysis_utils.load_in_df_with_progress(filePath = pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/all_impact_mutations_annotated_cohort.maf', nLinesFile = 275000)


In [9]:
allImpactMuts = maf_analysis_utils.fix_mll_genes(allImpactMuts)

In [23]:
poleStopGainRecurrentAlleles = set(['PIK3R1_p.R348*', 'TP53_p.R213*', 'ASXL2_p.R312*', 'FBXW7_p.R658*', 'SMAD4_p.E520*', 'CASP8_p.E239*', 'ASXL2_p.E45*', 'APC_p.R2237*', 'ARID1A_p.R1989*', 'APC_p.R1114*', 'MSH2_p.E580*', 'ATM_p.R250*', 'ARID1A_p.E896*', 'RASA1_p.E429*', 'NF1_p.R2450*'])
msiFrameshiftRecurrentAlleles = set(['ARID1A_p.Q758', 'SMO_p.P694', 'PTPRT_p.P1094', 'SMAD4_p.S32', 'HLA-B_p.P209', 'SOX9_p.P346', 'ARID1A_p.D1850', 'ZFHX3_p.E1892', 'TCF7L2_p.K468', 'MSH3_p.K383', 'CTCF_p.T204', 'AXIN2_p.E405', 'NSD1_p.M1531', 'MRE11A_p.N511', 'ERF_p.G299', 'KMT2B_p.G1879', 'MLL2_p.G1235', 'RNF43_p.R117', 'RNF43_p.G659', 'JAK1_p.K860', 'GLI1_p.G274', 'ARID1A_p.G276', 'ARID1A_p.F2141', 'B2M_p.L13', 'PDCD1_p.T36', 'CIC_p.P1248', 'KDM5A_p.G1200', 'PTEN_p.T321', 'ARID1A_p.Q372', 'IRS1_p.P368', 'PPM1D_p.N512', 'EP300_p.M1470', 'SOX9_p.V306', 'KMT2D_p.P2354', 'CD79A_p.R131', 'PTCH1_p.P1202', 'MSH6_p.F1088', 'APC_p.E1464', 'CREBBP_p.I1084', 'ARID5B_p.K967', 'KMT2D_p.V3089', 'ARID1A_p.T294', 'ARID1A_p.K1072', 'B2M_p.T93', 'SPRED1_p.D79', 'TGFBR2_p.K153', 'MLL2_p.P647', 'KMT2C_p.F4496', 'TCF7L2_p.L200', 'ARID1A_p.P224', 'FBXW7_p.S668', 'AXIN2_p.H661', 'ARAF_p.R255', 'ARID1A_p.P1326', 'PTEN_p.L265', 'B2M_p.V69', 'INPPL1_p.R1156', 'RPS6KA4_p.S772', 'SOX17_p.P328', 'PPM1D_p.L450', 'CARD11_p.R555', 'APC_p.E1554', 'AMER1_p.F173', 'NF1_p.Y628'])

In [10]:
#FIX MSI ALLELES
reload(msi_analysis_utils)
msiSummary = pd.read_table('/Users/friedman/Desktop/impact_microsatellites.txt')
allImpactMuts['allele'] = allImpactMuts.apply(lambda row: str(row['Hugo_Symbol']) + '_' + str(row['HGVSp_Short']), axis=1)
msiSummary['allele'] = msiSummary.apply(lambda row: str(row['Hugo_Symbol']) + '_' + str(row['HGVSp_Short']), axis=1)
neverObservedSites, msiSitesToNameMapping, mafMsiSiteToNameMapping =  msi_analysis_utils.standardize_allele_names(msiSummary, allImpactMuts)

allImpactMuts['correctedAllele'] = allImpactMuts['allele'].apply(lambda x:
                                                                 mafMsiSiteToNameMapping[x] if x in mafMsiSiteToNameMapping else None)



5.74382538771 percent done
11.4876507754 percent done
17.2314761631 percent done
22.9753015508 percent done
28.7191269385 percent done
34.4629523262 percent done
40.206777714 percent done
45.9506031017 percent done
51.6944284894 percent done
57.4382538771 percent done
63.1820792648 percent done
68.9259046525 percent done
74.6697300402 percent done
80.4135554279 percent done
86.1573808156 percent done
91.9012062033 percent done
97.645031591 percent done


In [11]:
impactSigs = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/signatures_from_unfiltered_maf.txt')
impactSigs = mutationSigUtils.merge_signature_columns(impactSigs)
impactSigs['dominantSignature'] = impactSigs.apply(lambda row: 
        mutationSigUtils.get_dominant_signature(row.to_dict(), cols=None, prefix='mean', notEnoughMuts= True), axis=1)

casesWithMSISignature = set(impactSigs[(impactSigs['dominantSignature'] == 'mean_MMR')
                                      |((impactSigs['Nmut_Mb'] >= 30) & (impactSigs['dominantSignature'].isin(set(['mean_1']))))
                                        ]['Tumor_Sample_Barcode'])

casesWithPOLESignature = set(impactSigs[(impactSigs['dominantSignature'] == 'mean_10') & (impactSigs['Nmut_Mb'] > 30)]['Tumor_Sample_Barcode'])

In [15]:
msiPoleMuts = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(casesWithMSISignature | casesWithPOLESignature)]
#alleleCounts = Counter(msiPoleMuts['allele'])


In [17]:
#THIS LINE TAKES A REALLY LONG TIME
msiPoleMuts['multipletPresentInCase'] = msiPoleMuts.apply(lambda row:
                is_allele_multiplet(msiPoleMuts, row['allele'], row['Tumor_Sample_Barcode']), axis=1)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
tumorSuppresors = set(['ERRFI1', 'ASXL2', 'PMAIP1', 'ACTG1', 'SUFU', 'FBXO11', 'MEN1', 'FAM58A', 'B2M', 'RB1', 'DUSP22', 'SESN1', 'GPS2', 'RAD51D', 'SMG1', 'CDC73', 'MAP3K1', 'SMARCB1', 'INPP4B', 'PARK2', 'SMAD4', 'CBFB', 'CDH1', 'PPP6C', 'SETDB1', 'SETDB2', 'NF2', 'CDKN2B', 'CDKN2C', 'CDKN2A', 'DDX3X', 'PIK3R1', 'BARD1', 'PDS5B', 'KLF4', 'SPRED1', 'VHL', 'SMAD2', 'PMS1', 'PMS2', 'SETD2', 'GATA3', 'TBL1XR1', 'MUTYH', 'SOCS1', 'FAM175A', 'ROBO1', 'ARID1B', 'ARID1A', 'TCF7L2', 'STK11', 'FOXA1', 'PTEN', 'FAT1', 'FAS', 'CYLD', 'MAX', 'SH2D1A', 'APC', 'NTHL1', 'CTCF', 'KDM5C', 'KMT2C', 'ZFHX3', 'FOXP1', 'PIGA', 'CDKN1B', 'CDKN1A', 'FUBP1', 'MSH2', 'ID3', 'TNFRSF14', 'TRAF3', 'EP400', 'BRIP1', 'ARID4A', 'ARID4B', 'XRCC2', 'DAXX', 'SDHAF2', 'ASXL1', 'AMER1', 'RASA1', 'EGR1', 'MST1', 'SOX17', 'RUNX1', 'PIK3R3', 'NCOR1', 'NF1', 'JAK1', 'PTPRD', 'CHEK2', 'CHEK1', 'SMC1A', 'TMEM127', 'STAG1', 'RAD51', 'TCF3', 'STAG2', 'ARID2', 'RAD50', 'RNF43', 'PARP1', 'BLM', 'CUX1', 'RECQL', 'RAD21', 'PTPN2', 'PTPN1', 'SLX4', 'INHA', 'PAX5', 'IRF1', 'TP53', 'HLA-A', 'IRF8', 'CBL', 'TOP1', 'SHQ1', 'PRDM1', 'NSD1', 'ATXN2', 'CREBBP', 'HDAC4', 'SESN2', 'PPP2R1A', 'EPHA7', 'ATM', 'EPHA3', 'POT1', 'SMAD3', 'MOB3B', 'TBX3', 'POLE', 'ATR', 'FANCD2', 'FH', 'BCORL1', 'SOX9', 'IKZF3', 'TSC1', 'TP63', 'MRE11A', 'SDHC', 'BTG1', 'POLD1', 'CIITA', 'SMC3', 'SAMHD1', 'RTEL1', 'ECT2L', 'PIK3R2', 'CRBN', 'FANCC', 'NBN', 'FANCA', 'HLA-B', 'RECQL4', 'DUSP4', 'ERCC2', 'FBXW7', 'TGFBR2', 'TGFBR1', 'MSH3', 'RBM15', 'TET1', 'TET3', 'SESN3', 'MGA', 'LTB', 'FOXL2', 'SH2B3', 'BCOR', 'HIST1H1D', 'ATRX', 'EP300', 'RAD51C', 'RAD51B', 'HIST1H1B', 'TNFAIP3', 'DICER1', 'ARID5B', 'LATS2', 'FOXO1', 'KEAP1', 'EZH2', 'SP140', 'NKX3-1', 'PBRM1', 'PALB2', 'CIC', 'BRCA1', 'DTX1', 'FLCN', 'SPEN', 'CD58', 'ERCC3', 'ERCC4', 'MSH6', 'BCL11B', 'BMPR1A', 'ERF', 'BRCA2', 'NOTCH2', 'EED', 'MITF', 'ELF3', 'SMARCA4', 'BBC3', 'ANKRD11', 'CEBPA', 'BCL2L11', 'AXIN2', 'AXIN1', 'CDK12', 'ESCO2', 'MLH1', 'SDHB', 'MED12', 'HNF1A', 'RYBP', 'ATP6V1B2', 'DNMT3B', 'KMT2B', 'KMT2A', 'DNMT3A', 'NFKBIA', 'TRAF5', 'KMT2D', 'SPOP', 'RBM10', 'P2RY8', 'TP53BP1', 'TSC2', 'KDM6A', 'EPCAM', 'PHOX2B', 'NPM1', 'BCL10', 'LATS1', 'HOXB13', 'ARID3A', 'PTPRT', 'PTPRS', 'INPPL1', 'NOTCH4', 'TET2', 'NOTCH1', 'CASP8', 'NOTCH3', 'GRIN2A', 'MAP2K4', 'WT1', 'BACH2', 'SDHA', 'BAP1', 'PTCH1', 'SDHD'])

In [25]:
msiPoleMuts['alleleClass'] = msiPoleMuts.apply(lambda row: 'recurrentPoleStopGain' if row['allele'] in poleStopGainRecurrentAlleles
                        else 'recurrentMsiIndel' if row['correctedAllele'] in msiFrameshiftRecurrentAlleles
                        else 'notRecurrentTSG' if row['Hugo_Symbol'] in tumorSuppresors else 'Oncogene', axis=1)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
#msiPoleMuts['multipletPresentInCase']

alleleCounts = dict(msiPoleMuts['allele'].value_counts())
multipletAlleleCounts = dict(msiPoleMuts[msiPoleMuts['multipletPresentInCase'] == 1]['allele'].value_counts())
alleleClassMapping = dict(zip(msiPoleMuts['allele'], msiPoleMuts['alleleClass']))

listOfDicts = []
cntr = 0
for allele in set(msiPoleMuts['allele']):
    cntr += 1
    if cntr%500 == 0:
        print cntr, len(set(msiPoleMuts['allele']))
    
    nAlleles = alleleCounts[allele]
    nMultiplet = 0
    if allele in multipletAlleleCounts: nMultiplet = multipletAlleleCounts[allele]
    fracMultiplet = 1.0*nMultiplet/nAlleles
    #alleleMaf = msiPoleMuts[msiPoleMuts['allele'] == allele]
    listOfDicts.append({'nOccurencesAllele': nAlleles, 'fracOccurencesMultiplet': fracMultiplet,
                       'nMultiplet':nMultiplet, 'allele':allele, 'alleleClass': alleleClassMapping[allele]})
    
df = pd.DataFrame(listOfDicts)

500 49865
1000 49865
1500 49865
2000 49865
2500 49865
3000 49865
3500 49865
4000 49865
4500 49865
5000 49865
5500 49865
6000 49865
6500 49865
7000 49865
7500 49865
8000 49865
8500 49865
9000 49865
9500 49865
10000 49865
10500 49865
11000 49865
11500 49865
12000 49865
12500 49865
13000 49865
13500 49865
14000 49865
14500 49865
15000 49865
15500 49865
16000 49865
16500 49865
17000 49865
17500 49865
18000 49865
18500 49865
19000 49865
19500 49865
20000 49865
20500 49865
21000 49865
21500 49865
22000 49865
22500 49865
23000 49865
23500 49865
24000 49865
24500 49865
25000 49865
25500 49865
26000 49865
26500 49865
27000 49865
27500 49865
28000 49865
28500 49865
29000 49865
29500 49865
30000 49865
30500 49865
31000 49865
31500 49865
32000 49865
32500 49865
33000 49865
33500 49865
34000 49865
34500 49865
35000 49865
35500 49865
36000 49865
36500 49865
37000 49865
37500 49865
38000 49865
38500 49865
39000 49865
39500 49865
40000 49865
40500 49865
41000 49865
41500 49865
42000 49865
42500 49865


In [35]:
df['orderingVal'] = df['alleleClass'].apply(lambda x: 1 if x == 'recurrentPoleStopGain' else 2 if x == 'recurrentMsiIndel'
                        else 3 if x == 'notRecurrentTSG' else 4)

In [36]:
df.to_csv('~/Desktop/WORK/dataForLocalPlotting/alleleRecurrences.tsv', index=False, sep='\t')

In [39]:
Counter(allImpactMuts['Strand'])

Counter({'+': 275171})